In [1]:
import json
from pprint import pprint as pprint
import pandas as pd
import requests
import gmaps
from config import gmapsapi
gmaps.configure(api_key=gmapsapi)

In [10]:
year = 1980
while year <= 2016:
    year += 1
    file = pd.read_csv(f"../aqi_by_year_by_month/{year}countyaqi.csv")
    filedf = pd.DataFrame(file)
    countynames = filedf["county Name"].unique()
    filedf.to_csv(f"{year}_county_aqi.csv")

In [3]:
latitude = []
longitude = []
for county in countynames:
    target_county = county
    target_url = (f"https://maps.googleapis.com/maps/api/geocode/json?address={target_county}")
    geo_data = requests.get(target_url).json()
    try:
        lat = geo_data["results"][0]["geometry"]["location"]["lat"]
        latitude.append(lat)
        lng = geo_data["results"][0]["geometry"]["location"]["lng"]
        longitude.append(lng)
    except IndexError:
        latitude.append("whoops")
        longitude.append("dope")

In [4]:
# county = "Albany"
# target_url = (f"https://maps.googleapis.com/maps/api/geocode/json?address={county}")
# geo_data = requests.get(target_url).json()
# geo_data["results"][0]["geometry"]["location"]["lat"]

In [5]:
filedf["Latitude"] = latitude
filedf["Longitude"] = longitude

In [6]:
filedf.head()

,county Name,01,02,03,04,05,Latitude,Longitude
0,Adams,30.065574,32.051282,31.182796,26.764706,NaN,36.743,-95.9117
1,Aiken,NaN,32.600000,41.666667,46.866667,NaN,33.5604,-81.7196
2,Alameda,60.451613,50.285714,NaN,NaN,NaN,37.7652,-122.242
3,Alamosa,29.827586,15.592593,19.774194,36.300000,NaN,whoops,dope
4,Albany,44.822581,43.017857,34.850000,NaN,NaN,whoops,dope


In [34]:
finaldf = filedf[filedf.Latitude != "whoops"]


In [82]:
coordinates = finaldf[["Latitude", "Longitude"]].astype(float)
center = ["39.8283","-98.5795"]
fig = gmaps.figure(zoom_level= 4, center = center)

In [83]:

heatmaplayer = gmaps.heatmap_layer(coordinates, weights = finaldf["01"],
                                   max_intensity = filedf["01"].max(), point_radius = 1.0)
fig.add_layer(heatmaplayer)
fig

Figure(layout=FigureLayout(height='420px'))